In [11]:
import PyPDF2
import minsearch
import nltk
from nltk.tokenize import sent_tokenize

In [32]:
import nltk

# Remove NLTK's downloaded files to avoid incorrect resources
nltk.data.clear_cache()

# Ensure that the path is correct
nltk.data.path.append('/Users/monahaghighi/nltk_data')

# Download the correct punkt resource again
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/monahaghighi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
### Step 1: Extract Text from the PDF

In [33]:
# Extract text from the PDF
def extract_text_from_pdf(pdf_path):
    with open("Renter's Insurance.pdf", 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        policy_text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            policy_text += text + "\n"
    return policy_text

policy_text = extract_text_from_pdf("Renter's Insurance.pdf")

In [34]:
policy_text

'CONTINUE TO THE NEXT PAGE\nFOR A GUIDE TO YOUR\nRENTERS ENDORSEMENT\nPACKET40000007H4129197624975000000*000001*\nLiberty Mutual\nPO Box 958416\nLake Mary, FL 32795-9959\nZohreh Motemanni Kenari\nNaser Mirzaei Mehmandoust Sofla\n351 N Post Oak Ln 712\nHouston TX 77024-5953\nCONTACT US\nQuestions About\nYour Policy\nBy Phone\nFor service:\nMon - Fri 8AM-10PM EST\nSat 8AM-8PM EST\nSun 11AM-5PM EST\nVisit Us Online\nLibertyMutual.com\nBy Phone\n1-800-2CLAIMS\n(1-800-225-2467)To Report a Claim\nOnline\nLibertyMutual.com/Claims\nSign Up for eService\nLibertyMutual.com/\nregister-Pay your bill\n-Go paperless\n-View your policy\n-File or view a claim\nManage your policy 24/7 atThank you for insuring with Liberty Mutual.\nThis package contains your updated policy declarations and documents that\nreflect changes made to your policy.\n• • Change Name Zohreh Motemanni\nKenariAdd Inflation Protection\n•Change Scheduled Property\nTerritory\nPlease look over this information and keep it with your in

In [35]:
### Step 2: Split Text into Chunks

In [45]:
import nltk

# Add your preferred path where nltk should download the data
nltk.data.path.append('/Users/monahaghighi/nltk_data')

# Force download the punkt resource
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/monahaghighi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [46]:
import os
punkt_path = '/Users/monahaghighi/nltk_data/tokenizers/punkt'
print("Punkt available:", os.path.exists(punkt_path))

Punkt available: True


In [47]:
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

# Load the English tokenizer manually
english_tokenizer = PunktSentenceTokenizer()

# Now use it to tokenize your policy text
sentences = english_tokenizer.tokenize(policy_text)

In [48]:
sentences[0]

'CONTINUE TO THE NEXT PAGE\nFOR A GUIDE TO YOUR\nRENTERS ENDORSEMENT\nPACKET40000007H4129197624975000000*000001*\nLiberty Mutual\nPO Box 958416\nLake Mary, FL 32795-9959\nZohreh Motemanni Kenari\nNaser Mirzaei Mehmandoust Sofla\n351 N Post Oak Ln 712\nHouston TX 77024-5953\nCONTACT US\nQuestions About\nYour Policy\nBy Phone\nFor service:\nMon - Fri 8AM-10PM EST\nSat 8AM-8PM EST\nSun 11AM-5PM EST\nVisit Us Online\nLibertyMutual.com\nBy Phone\n1-800-2CLAIMS\n(1-800-225-2467)To Report a Claim\nOnline\nLibertyMutual.com/Claims\nSign Up for eService\nLibertyMutual.com/\nregister-Pay your bill\n-Go paperless\n-View your policy\n-File or view a claim\nManage your policy 24/7 atThank you for insuring with Liberty Mutual.'

In [50]:
# Combine sentences into chunks (e.g., 5 sentences per chunk)
def create_chunks(sentences, chunk_size=5):
    chunks = []
    for i in range(0, len(sentences), chunk_size):
        chunk = ' '.join(sentences[i:i+chunk_size])
        chunks.append(chunk)
    return chunks

chunks = create_chunks(sentences)

In [51]:
len(chunks)

38

In [52]:
import openai
import os

# Set your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')  # Make sure to set your API key in the environment

In [53]:
openai.api_key

In [56]:
!pip install openai==0.28

  Attempting uninstall: openai
    Found existing installation: openai 1.50.2
    Uninstalling openai-1.50.2:
      Successfully uninstalled openai-1.50.2


In [58]:
import openai

In [60]:
def get_embedding(text, model="text-embedding-ada-002"):
    response = openai.Embedding.create(
        input=text,
        model=model
    )
    return response['data'][0]['embedding']

# Generate embeddings for all chunks
chunk_embeddings = []
for chunk in chunks:
    embedding = get_embedding(chunk)
    chunk_embeddings.append({'chunk': chunk, 'embedding': embedding})


APIRemovedInV1: 

You tried to access openai.Embedding, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [62]:
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 5.6 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 4.3 MB/s eta 0:00:00 MB/s eta 0:00:01


In [63]:
from transformers import RagTokenizer, RagSequenceForGeneration

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
